In [2]:

from google.cloud import bigquery
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

client = bigquery.Client()
dataset_id = 'weather_data'

dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"
dataset = client.create_dataset(dataset)  # Make an API request.

print("Dataset {} created.".format(dataset.dataset_id))


Dataset weather_data created.


In [3]:

dataset_id = 'weather_data'
table_id = 'weather_data_tb'

# Construct a reference to the "my_dataset" dataset
dataset_ref = client.dataset(dataset_id)

# Construct a BigQuery table object
table_ref = dataset_ref.table(table_id)
table = bigquery.Table(table_ref)

# Configure the load job.
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    skip_leading_rows=1,
    # The source format defaults to CSV, so the property is optional.
    source_format=bigquery.SourceFormat.CSV,
    write_disposition="WRITE_TRUNCATE"
)


In [4]:

# Specify the Cloud Storage URI of the file to load.
uri = "gs://labs.roitraining.com/data-to-ai-workshop/weather_data.csv"

# Start the load job.
load_job = client.load_table_from_uri(
    uri, table, job_config=job_config
)

load_job.result()

LoadJob<project=qwiklabs-gcp-04-14e3a67e50ad, location=US, id=d36b1714-bdf2-4eca-83c8-522f4b6cbfdc>

In [8]:
query_1 = """SELECT * FROM `weather_data.weather_data_tb`"""

weather_data_df = client.query(query_1).to_dataframe()
weather_data_df.head()

,date,city,state,temperature_f,wind_speed_mph,precipitation_in,barometric_pressure_inHg,humidity_percent,weather_condition
0,2025-02-21,Atlanta,GA,55.7,5.0,0.12,29.80,50.4,Cloudy
1,2025-02-26,Atlanta,GA,75.2,10.4,0.03,29.58,49.9,Cloudy
2,2025-03-01,Atlanta,GA,51.7,4.7,0.08,29.74,49.9,Cloudy
3,2025-03-05,Atlanta,GA,74.4,5.1,0.02,29.92,50.4,Cloudy
4,2025-03-10,Atlanta,GA,59.5,9.6,0.09,29.67,57.2,Cloudy


In [9]:
weather_data_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      300 non-null    dbdate 
 1   city                      300 non-null    object 
 2   state                     300 non-null    object 
 3   temperature_f             300 non-null    float64
 4   wind_speed_mph            300 non-null    float64
 5   precipitation_in          300 non-null    float64
 6   barometric_pressure_inHg  300 non-null    float64
 7   humidity_percent          300 non-null    float64
 8   weather_condition         300 non-null    object 
dtypes: dbdate(1), float64(5), object(3)
memory usage: 21.2+ KB


In [10]:
weather_data_df.describe()

,temperature_f,wind_speed_mph,precipitation_in,barometric_pressure_inHg,humidity_percent
count,300.000000,300.000000,300.000000,300.000000,300.000000
mean,53.418333,14.589667,0.930033,29.501267,63.374667
std,20.068146,13.023698,1.033668,0.462938,21.246583
min,14.300000,0.000000,0.000000,28.530000,20.100000
25%,37.675000,5.075000,0.060000,29.120000,44.950000
50%,53.100000,10.300000,0.605000,29.555000,67.500000
75%,68.175000,18.400000,1.490000,29.850000,81.200000
max,94.400000,49.800000,3.900000,30.290000,98.600000


In [13]:
# Get unique values for each column as a dictionary
unique_per_col = {col: weather_data_df[col].unique() for col in weather_data_df.columns}

for col, values in unique_per_col.items():
    print(f"{col}: {values}")

date: <DateArray>
[datetime.date(2025, 2, 21), datetime.date(2025, 2, 26),
  datetime.date(2025, 3, 1),  datetime.date(2025, 3, 5),
 datetime.date(2025, 3, 10), datetime.date(2025, 3, 14),
 datetime.date(2025, 2, 19),  datetime.date(2025, 3, 9),
 datetime.date(2025, 3, 13), datetime.date(2025, 3, 19),
 datetime.date(2025, 2, 23), datetime.date(2025, 2, 25),
 datetime.date(2025, 2, 27),  datetime.date(2025, 3, 3),
 datetime.date(2025, 2, 24), datetime.date(2025, 3, 15),
 datetime.date(2025, 3, 17),  datetime.date(2025, 3, 6),
 datetime.date(2025, 3, 11), datetime.date(2025, 3, 12),
  datetime.date(2025, 3, 2),  datetime.date(2025, 3, 8),
 datetime.date(2025, 3, 16), datetime.date(2025, 3, 18),
 datetime.date(2025, 2, 18),  datetime.date(2025, 3, 4),
 datetime.date(2025, 2, 22), datetime.date(2025, 2, 28),
  datetime.date(2025, 3, 7), datetime.date(2025, 2, 20)]
Length: 30, dtype: dbdate
city: ['Atlanta' 'Boston' 'Chicago' 'Denver' 'Houston' 'Los Angeles' 'Miami'
 'New York' 'San Francis

In [11]:
print(weather_data_df.isnull().sum())

date                        0
city                        0
state                       0
temperature_f               0
wind_speed_mph              0
precipitation_in            0
barometric_pressure_inHg    0
humidity_percent            0
weather_condition           0
dtype: int64


## **CREATE MODEL statement to add Gemini to project**

In [16]:
%%bigquery

CREATE OR REPLACE MODEL `weather_data.gemini_weather_model`
REMOTE WITH CONNECTION DEFAULT
OPTIONS(ENDPOINT = 'gemini-2.0-flash-001');

Query is running:   0%|          |

""


In [34]:
%%bigquery

Select ml_generate_text_llm_result as weather_report,
*
FROM ML.GENERATE_TEXT(
  MODEL `weather_data.gemini_weather_model`,
  (
    Select
      CONCAT('Based on the following weather data, create a well structured weather report or warning if conditions are severe:',
      'Location: ', city, ', ', state,
      'Date: ', CAST(date AS STRING),
      'Temperature: ', CAST(temperature_f AS STRING), ' degrees Fahrenheit',
      'wind speed: ', CAST(wind_speed_mph AS STRING), ' mph',
      'precipitation: ', CAST(precipitation_in AS STRING), ' in',
      'barometric pressure: ', CAST(barometric_pressure_inHg AS STRING), ' inches of mercury',
      'humidity: ', CAST(humidity_percent AS STRING), ' percent',
      'weather condition: ', weather_condition
      ) AS prompt,*
      FROM `weather_data.weather_data_tb` limit 1000
  ),
  STRUCT(
  0.2 as TEMPERATURE,
  256 as MAX_OUTPUT_TOKENS,
  TRUE as flatten_json_output
))


Query is running:   0%|          |

Downloading:   0%|          |

,weather_report,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt,date,city,state,temperature_f,wind_speed_mph,precipitation_in,barometric_pressure_inHg,humidity_percent,weather_condition
0,Here's a weather report based on the provided ...,Here's a weather report based on the provided ...,None,,"Based on the following weather data, create a ...",2025-03-19,Boston,MA,60.7,6.4,0.04,29.83,49.4,Cloudy
1,Here's a weather report based on the provided ...,Here's a weather report based on the provided ...,None,,"Based on the following weather data, create a ...",2025-03-12,Houston,TX,59.5,7.1,0.11,29.64,42.4,Cloudy
2,"## Houston, TX Weather Report - March 13, 2025...","## Houston, TX Weather Report - March 13, 2025...",None,,"Based on the following weather data, create a ...",2025-03-13,Houston,TX,74.4,10.8,0.12,29.76,44.1,Cloudy
3,"## Miami, FL Weather Report - March 13, 2025\n...","## Miami, FL Weather Report - March 13, 2025\n...",None,,"Based on the following weather data, create a ...",2025-03-13,Miami,FL,55.2,8.0,0.05,29.75,59.9,Cloudy
4,"## New York, NY Weather Report - March 12, 202...","## New York, NY Weather Report - March 12, 202...",None,,"Based on the following weather data, create a ...",2025-03-12,New York,NY,65.9,11.2,0.02,29.93,53.3,Cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"## Atlanta, GA Weather Report - March 18, 2025...","## Atlanta, GA Weather Report - March 18, 2025...",None,,"Based on the following weather data, create a ...",2025-03-18,Atlanta,GA,91.0,4.0,0.01,29.91,39.4,Sunny
296,"## Houston, TX Weather Report - March 8, 2025\...","## Houston, TX Weather Report - March 8, 2025\...",None,,"Based on the following weather data, create a ...",2025-03-08,Houston,TX,83.5,5.6,0.03,30.29,24.4,Sunny
297,"## Los Angeles Weather Report - February 25, 2...","## Los Angeles Weather Report - February 25, 2...",None,,"Based on the following weather data, create a ...",2025-02-25,Los Angeles,CA,84.6,2.6,0.02,30.03,28.6,Sunny
298,"## Los Angeles Weather Report - March 18, 2025...","## Los Angeles Weather Report - March 18, 2025...",None,,"Based on the following weather data, create a ...",2025-03-18,Los Angeles,CA,75.7,1.9,0.02,30.25,26.4,Sunny


## **Add the weather report to the table.**

In [57]:
%%bigquery

CREATE OR REPLACE TABLE
`weather_data.weather_reports` AS (
    Select ml_generate_text_llm_result  as weather_report,
*
FROM ML.GENERATE_TEXT(
  MODEL `weather_data.gemini_weather_model`,
  (
    Select
      CONCAT('Based on the following weather data, create a well structured weather report or warning if conditions are severe:',
      'Location: ', city, ', ', state,
      'Date: ', CAST(date AS STRING),
      'Temperature: ', CAST(temperature_f AS STRING), ' F',
      'wind speed: ', CAST(wind_speed_mph AS STRING), ' mph',
      'precipitation: ', CAST(precipitation_in AS STRING), ' in',
      'humidity: ', CAST(humidity_percent AS STRING), ' %',
      'weather condition: ', weather_condition
      ) AS prompt, *
      FROM `weather_data.weather_data_tb`
  ),
  STRUCT(
  0.2 as TEMPERATURE,
  TRUE as flatten_json_output
)
)
)


Query is running:   0%|          |

""


In [58]:
%%bigquery
SELECT * FROM `weather_data.weather_reports` limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,weather_report,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt,date,city,state,temperature_f,wind_speed_mph,precipitation_in,barometric_pressure_inHg,humidity_percent,weather_condition
0,"## Atlanta, GA Weather Report - February 26, 2...","## Atlanta, GA Weather Report - February 26, 2...",None,,"Based on the following weather data, create a ...",2025-02-26,Atlanta,GA,75.2,10.4,0.03,29.58,49.9,Cloudy
1,"## Atlanta, GA Weather Report - February 21, 2...","## Atlanta, GA Weather Report - February 21, 2...",None,,"Based on the following weather data, create a ...",2025-02-21,Atlanta,GA,55.7,5.0,0.12,29.80,50.4,Cloudy
2,"Okay, here's a weather report based on the pro...","Okay, here's a weather report based on the pro...",None,,"Based on the following weather data, create a ...",2025-03-10,Atlanta,GA,59.5,9.6,0.09,29.67,57.2,Cloudy
3,"Okay, here's a weather report based on the pro...","Okay, here's a weather report based on the pro...",None,,"Based on the following weather data, create a ...",2025-03-05,Atlanta,GA,74.4,5.1,0.02,29.92,50.4,Cloudy
4,"Okay, here's a weather report based on the pro...","Okay, here's a weather report based on the pro...",None,,"Based on the following weather data, create a ...",2025-03-14,Atlanta,GA,71.7,7.2,0.18,29.92,55.3,Cloudy
5,"## Atlanta, GA Weather Report - March 1, 2025\...","## Atlanta, GA Weather Report - March 1, 2025\...",None,,"Based on the following weather data, create a ...",2025-03-01,Atlanta,GA,51.7,4.7,0.08,29.74,49.9,Cloudy
6,"## Boston, MA Weather Report - March 9, 2025\n...","## Boston, MA Weather Report - March 9, 2025\n...",None,,"Based on the following weather data, create a ...",2025-03-09,Boston,MA,76.7,4.3,0.09,29.52,40.9,Cloudy
7,"## Boston, MA Weather Report - March 19, 2025\...","## Boston, MA Weather Report - March 19, 2025\...",None,,"Based on the following weather data, create a ...",2025-03-19,Boston,MA,60.7,6.4,0.04,29.83,49.4,Cloudy
8,"Okay, here's a weather report based on the pro...","Okay, here's a weather report based on the pro...",None,,"Based on the following weather data, create a ...",2025-03-13,Boston,MA,71.9,9.8,0.16,29.99,42.3,Cloudy
9,"## Boston Weather Report - February 19, 2025\n...","## Boston Weather Report - February 19, 2025\n...",None,,"Based on the following weather data, create a ...",2025-02-19,Boston,MA,61.7,3.9,0.11,29.62,54.1,Cloudy
